In [1]:
import torch
import torch.distributions as dists
import numpy as np
import helper.wideresnet as wrn
import helper.dataloaders as dl
from helper import util
from netcal.metrics import ECE

from laplace import Laplace
import json
import torch.nn as nn

# np.random.seed(7777)
# torch.manual_seed(7777)
# torch.backends.cudnn.deterministic = True
# torch.backends.cudnn.benchmark = True

# train_loader_dl = dl.CIFAR10(train=True)
# test_loader_dl = dl.CIFAR10(train=False)
# targets = torch.cat([y for x, y in test_loader_dl], dim=0).numpy()

/home/qitian/anaconda3/envs/anomaly_seg2/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Files already downloaded and verified
Files already downloaded and verified


In [2]:
import sys
sys.path.insert(0, "/home/qitian/Documents/Anomaly_detection_semseg/training_logs") # path to model
sys.path.insert(0, "/home/qitian/Documents/Anomaly_detection_semseg/") # path to github

In [3]:
from utils.evaluate import evaluate_ResNet101_MCdropout_model as evaluate_model
from models.resnet101_mcdropout import get_model

In [4]:
param_path = '/home/qitian/Documents/Anomaly_detection_semseg/training_logs/resnet101_mcdropout_seed42_0/params.json' # path to params
with open(param_path, 'r') as f:
    params = json.load(f)

In [5]:
model_id = "resnet101_mcdropout_seed42"
model_i = 0
num_steps = 40000

In [6]:
restore_from = "/home/qitian/Documents/Anomaly_detection_semseg/training_logs/%s_%d/checkpoint_%d.pth" % (model_id, model_i, num_steps)

In [7]:
restore_from

'/home/qitian/Documents/Anomaly_detection_semseg/training_logs/resnet101_mcdropout_seed42_0/checkpoint_40000.pth'

In [8]:
from utils.datasets import DatasetCityscapesEval
from torch.utils import data
from utils.datasets import DatasetCityscapesAugmentation

data_dir = params["data_dir"]
data_list = params["val_list"]
batch_size = params["batch_size"]
num_classes = params["num_classes"]
snapshot_dir = params["snapshot_dir"]
crop_size = params["crop_size"]
num_workers = batch_size
batch_size = 4

eval_dataset = DatasetCityscapesEval(root=data_dir, list_path=data_list)
eval_loader = data.DataLoader(eval_dataset, batch_size=batch_size, shuffle=False, pin_memory=True, num_workers=num_workers)
train_dataset = DatasetCityscapesAugmentation(root=data_dir, list_path=data_list, max_iters=params["train_size"], crop_size=crop_size)
train_loader = data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)

DatasetCityscapesEval - num examples: 500
DatasetCityscapesAugmentation - num unique examples: 500
DatasetCityscapesAugmentation - num examples: 320000


In [15]:
device = "cuda:0"
model_raw = get_model(**params)
model_raw.load_state_dict(torch.load(restore_from))
model = nn.DataParallel(model_raw) if device == "gpu" else model_raw
model.train()

resnet101_mcdropout.py


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu2): ReLU()
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU()
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=True)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (

In [10]:
# The model is a standard WideResNet 16-4
# Taken as is from https://github.com/hendrycks/outlier-exposure
# model = wrn.WideResNet(16, 4, num_classes=10).cpu().eval()

# util.download_pretrained_model()
# model.load_state_dict(torch.load('./temp/CIFAR10_plain.pt'))

<All keys matched successfully>

In [20]:
class train_loader_wrapper:
    def __init__(self, train_loader):
        self.dataset=range(1000)
    def __iter__(self):
        while True:
            images, labels, _, _ = next(iter(train_loader))
            yield images.to(device), labels.to(device)

In [21]:
train_loader_wrapped = train_loader_wrapper(train_loader)

In [ ]:
# Error Training Here
# Tested methods： all / subnetwork / last_layer

In [27]:
la = Laplace(model, 'classification',
             subset_of_weights='all',
             hessian_structure='diag')
la.fit(train_loader_wrapped)
la.optimize_prior_precision(method='marglik')

RuntimeError: size mismatch (got input: [4, 19, 65, 65] , target: [4, 512, 512]

In [ ]:
@torch.no_grad()
def predict(dataloader, model, laplace=False):
    py = []

    for x, _, _, _ in dataloader:
        if laplace:
            py.append(model(x))
        else:
            py.append(torch.softmax(model(x.cuda()), dim=-1))

    return torch.cat(py).cpu().numpy()

In [ ]:
probs_laplace = torch.tensor(predict(eval_loader, la, laplace=True))
acc_laplace = (probs_laplace.argmax(-1) == targets).float().mean()
ece_laplace = ECE(bins=15).measure(probs_laplace.numpy(), targets.numpy())
nll_laplace = -dists.Categorical(probs_laplace).log_prob(targets).mean()

print(f'[Laplace] Acc.: {acc_laplace:.1%}; ECE: {ece_laplace:.1%}; NLL: {nll_laplace:.3}')

In [4]:
# targets = torch.tensor(targets)
# probs_map = torch.tensor(predict(test_loader, model, laplace=False))
# acc_map = (probs_map.argmax(-1) == targets).float().mean()
# ece_map = ECE(bins=15).measure(probs_map.numpy(), targets.numpy())
# nll_map = -dists.Categorical(probs_map).log_prob(targets).mean()

# print(f'[MAP] Acc.: {acc_map:.1%}; ECE: {ece_map:.1%}; NLL: {nll_map:.3}')

[MAP] Acc.: 94.8%; ECE: 2.0%; NLL: 0.172


In [ ]:
# la = Laplace(model, 'classification',
#              subset_of_weights='last_layer',
#              hessian_structure='kron')
# la.fit(train_loader)
# la.optimize_prior_precision(method='marglik')

In [6]:
# probs_laplace = torch.tensor(predict(test_loader, la, laplace=True))
# acc_laplace = (probs_laplace.argmax(-1) == targets).float().mean()
# ece_laplace = ECE(bins=15).measure(probs_laplace.numpy(), targets.numpy())
# nll_laplace = -dists.Categorical(probs_laplace).log_prob(targets).mean()

# print(f'[Laplace] Acc.: {acc_laplace:.1%}; ECE: {ece_laplace:.1%}; NLL: {nll_laplace:.3}')

[Laplace] Acc.: 94.8%; ECE: 0.8%; NLL: 0.157
